In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras import backend as K
from matplotlib import pyplot
import os
!pip install pypianoroll
import pypianoroll


     |████████████████████████████████| 5.6MB 10.1MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591954 sha256=e1a699b9740ab3039440c0152d2e470d2e3b9b0fa1e644292fdc39b02b85035d
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [ ]:
dataset_name = 'dataset_np_2.0_rearanged.npy'
path = '/content/gdrive/MyDrive/Colab Notebooks/senior_research/version 3.0/'
path_progression = path+'GAN_progression/'
path_model_backups = path+'GAN_models/'
path_dataset = '/content/gdrive/MyDrive/Colab Notebooks/senior_research/'+dataset_name

In [ ]:


def load_real_samples():

    X = np.load(path_dataset)
    return X


dataset = load_real_samples()

In [ ]:
def define_discriminator(in_shape=(768,128,2)):
    dropout = 0.3

    model = Sequential()
    model.add(Conv2D(128, (5,5), strides=(2, 1), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(64, (5,5), strides=(1, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(32, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(32, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(32, (5,5), strides=(2, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(16, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(16, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    # model.add(Conv2D(16, (5,5), strides=(1, 2), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))
    # model.add(Conv2DTranspose(8, (5,5), strides=(1, 1), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))
    # model.add(Conv2DTranspose(8, (5,5), strides=(1, 1), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))
    # model.add(Dropout(dropout))

    # model.add(Conv2D(8, (5,5), strides=(2, 1), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))
    # model.add(Conv2DTranspose(4, (5,5), strides=(1, 1), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))
    # model.add(Conv2DTranspose(4, (5,5), strides=(1, 1), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))
    # model.add(Dropout(dropout))

    model.add(Conv2D(4, (5,5), strides=(1, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.000002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    model = Sequential()

    n_nodes = 96 * 32 * 16
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((96, 32, 16)))


    channels_layer_1 = 256
    model.add(Conv2DTranspose(channels_layer_1, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_1, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_1, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    channels_layer_2 = 128
    model.add(Conv2DTranspose(channels_layer_2, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_2, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_2, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    channels_layer_3 = 64
    model.add(Conv2DTranspose(channels_layer_3, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_3, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_3, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    channels_layer_4 = 32
    model.add(Conv2DTranspose(channels_layer_4, (5,5), strides=(1,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_4, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_4, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    channels_layer_5 = 16
    model.add(Conv2DTranspose(channels_layer_5, (5,5), strides=(2,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_5, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_5, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    #at this point we have the desired size of (768,128), we just need to cut down the channels to 2
    channels_layer_6 = 8
    model.add(Conv2DTranspose(channels_layer_6, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_6, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_6, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    channels_layer_7 = 4
    model.add(Conv2DTranspose(channels_layer_7, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_7, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_7, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    channels_layer_8 = 2
    model.add(Conv2DTranspose(channels_layer_8, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(channels_layer_8, (5,5), strides=(1,1), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(channels_layer_8, (5,5), strides=(1,1), activation='sigmoid', padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.001, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected midis
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples) 
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)#[:,:,:,:,0]
    # X = X*128
    # X = np.floor(np.where(X < 30, 0, X))
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images (reversed grayscale)
def save_midis(examples, epoch, n=1):
    # save midis
    midis = examples*128
    midis = np.floor(np.where(midis < 1, 0, midis))

    midis=np.swapaxes(midis,1,3)
    midis=np.swapaxes(midis,2,3)

    #midis=examples
    for i in range(n):
        name = 'generated_midi_e%03d_n%03d' % (epoch+1, i+1)
        #initialize
        multitrack = pypianoroll.Multitrack(name=name)
        #create left and right tracks
        right_track = pypianoroll.StandardTrack(name='right',pianoroll=midis[i,0])
        left_track = pypianoroll.StandardTrack(name='left',pianoroll=midis[i,1])
        #add tracks
        multitrack.append(right_track)
        multitrack.append(left_track)
        #save
        pypianoroll.write(path_progression+name+'.mid',multitrack)

# size of the latent space
latent_dim = 1000
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)



In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # manually enumerate epochs
    for i in range(bat_per_epo):
        # enumerate batches over the training set
        g_d_loss_ratio = 1
        count = 0
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # update discriminator model weights

            #if count % int(g_d_loss_ratio)*3 == 0:
            if True:
            #if j%5==0:
                d_loss, _ = d_model.train_on_batch(X, y)
                g_d_loss_ratio = g_loss / d_loss
                if g_d_loss_ratio < 1:
                    g_d_loss_ratio = 1
                count=0
                #d_loss, _ = d_model.train_on_batch(X[:,:,:,:,np.newaxis], y)
                # summarize loss on this batch
                print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
                # evaluate the model performance, sometimes
                if j == 0:
                    losses[i,0] = d_loss
                    losses[i,1] = g_loss
            count+=1
        if (i+1) % 1 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=10, n_samples_save=1):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save midis
    save_midis(x_fake, epoch, n_samples_save)
    # save the generator and discriminator models
    g_filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(os.path.join(path_model_backups,g_filename))
    d_filename = 'discriminator_model_%03d.h5' % (epoch + 1)
    d_model.save(os.path.join(path_model_backups,d_filename))

n_epochs=500
losses = np.zeros((n_epochs,2))
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=n_epochs, n_batch=10)

Streaming output truncated to the last 5000 lines.
>28, 8/398, d=0.065, g=435.277
>28, 9/398, d=0.065, g=463.223
>28, 10/398, d=0.000, g=399.972
>28, 11/398, d=0.065, g=410.705
>28, 12/398, d=0.561, g=392.090
>28, 13/398, d=0.000, g=417.940
>28, 14/398, d=0.000, g=447.399
>28, 15/398, d=0.065, g=430.802
>28, 16/398, d=0.000, g=420.660
>28, 17/398, d=0.000, g=393.058
>28, 18/398, d=0.000, g=472.384
>28, 19/398, d=0.000, g=428.112
>28, 20/398, d=0.000, g=371.431
>28, 21/398, d=0.000, g=340.044
>28, 22/398, d=0.000, g=438.164
>28, 23/398, d=0.000, g=433.085
>28, 24/398, d=0.000, g=415.448
>28, 25/398, d=0.000, g=391.858
>28, 26/398, d=0.000, g=391.940
>28, 27/398, d=0.000, g=453.806
>28, 28/398, d=0.000, g=396.374
>28, 29/398, d=0.000, g=448.770
>28, 30/398, d=0.000, g=427.957
>28, 31/398, d=0.000, g=447.296
>28, 32/398, d=0.000, g=400.568
>28, 33/398, d=0.000, g=384.431
>28, 34/398, d=0.000, g=408.703
>28, 35/398, d=0.000, g=426.696
>28, 36/398, d=0.065, g=436.094
>28, 37/398, d=0.000, g

In [ ]:
def save_midis(examples, epoch, n=1):
    # save midis
    midis = examples*128
    midis = np.floor(np.where(midis < 1, 0, midis))
    for i in range(n):
        name = 'generated_midi_e%03d_n%03d' % (epoch+1, i+1)
        #initialize
        multitrack = pypianoroll.Multitrack(name=name)
        #create left and right tracks
        right_track = pypianoroll.StandardTrack(name='right',pianoroll=midis[i,0])
        left_track = pypianoroll.StandardTrack(name='left',pianoroll=midis[i,1])
        #add tracks
        multitrack.append(right_track)
        multitrack.append(left_track)
        #save
        pypianoroll.write(path_progression+name+'.mid',multitrack)
X_fake, y_fake = generate_fake_samples(g_model, 100, 1)
print(X_fake.shape)
save_midis(X_fake,0,1)
np.save('test',X_fake)

(1, 2, 1024, 128)
